In [7]:
import os
import re
import json
import time
import argparse
from typing import Dict, Any, List, Tuple, TypedDict

from dotenv import load_dotenv
load_dotenv()

True

In [8]:
import os
os.environ["PPLX_API_KEY"] = "your_perplexity_api_key"
os.environ["ALPHA_VANTAGE_API_KEY"] = "your_alpha_vantage_api_key"  # optional

PPLX_API_KEY = "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"
ALPHA_VANTAGE_API_KEY = "Z20QYDTQWVJB2VJO" 

In [9]:
import yfinance as yf
from alpha_vantage.foreignexchange import ForeignExchange

# Stock price lookup
def get_stock_price(symbol: str) -> str:
    try:
        stock = yf.Ticker(symbol)
        price = stock.history(period="1d")["Close"].iloc[-1]
        return f"Current price of {symbol} is ${price:.2f}"
    except Exception as e:
        return f"Error fetching stock data: {str(e)}"

# Forex price lookup
def get_forex_price(pair: str) -> str:
    try:
        fx = ForeignExchange(key=os.environ["ALPHA_VANTAGE_API_KEY"])
        data, _ = fx.get_currency_exchange_rate(from_currency=pair[:3], to_currency=pair[3:])
        return f"Exchange rate {pair[:3]}/{pair[3:]} is {data['5. Exchange Rate']}"
    except Exception as e:
        return f"Error fetching forex data: {str(e)}"

# Simple in-memory conversation storage
conversation_memory = []
def add_to_memory(role, content):
    conversation_memory.append({"role": role, "content": content})


In [ ]:
from openai import OpenAI

client = OpenAI(api_key=os.environ["PPLX_API_KEY"], base_url="https://api.perplexity.ai")

def phase1_chatbot():
    print("🤖 Finance Chatbot (Phase 1) – type 'exit' to quit")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            break

        # Tool calls
        if user_input.upper().startswith("STOCK "):
            symbol = user_input.split()[1]
            response = get_stock_price(symbol)
        elif user_input.upper().startswith("FOREX "):
            pair = user_input.split()[1]
            response = get_forex_price(pair)
        else:
            add_to_memory("user", user_input)
            completion = client.chat.completions.create(
                model="perplexity-chat",
                messages=conversation_memory + [{"role": "user", "content": user_input}],
                temperature=0.2
            )
            response = completion.choices[0].message.content
            add_to_memory("assistant", response)

        print(f"Bot: {response}\n")


In [ ]:
from langgraph.graph import StateGraph, END

def phase2_agent():
    def assistant_node(state):
        query = state["query"]
        if query.upper().startswith("STOCK "):
            return {"response": get_stock_price(query.split()[1])}
        elif query.upper().startswith("FOREX "):
            return {"response": get_forex_price(query.split()[1])}
        else:
            completion = client.chat.completions.create(
                model="perplexity-chat",
                messages=[{"role": "user", "content": query}],
                temperature=0.2
            )
            return {"response": completion.choices[0].message.content}

    # Build LangGraph
    builder = StateGraph(dict)
    builder.add_node("assistant", assistant_node)
    builder.set_entry_point("assistant")
    builder.add_edge("assistant", END)
    graph = builder.compile()

    print("🤖 Finance Agent (Phase 2) – type 'exit' to quit")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            break
        result = graph.invoke({"query": user_input})
        print(f"Bot: {result['response']}\n")


In [12]:
MODE = "phase1"   # change to "phase2" for LangGraph agent

if MODE == "phase1":
    phase1_chatbot()
elif MODE == "phase2":
    phase2_agent()


🤖 Finance Chatbot (Phase 1) – type 'exit' to quit
Bot: Exchange rate USD/INR is 87.48800000



TypeError: Missing required arguments; Expected either ('messages' and 'model') or ('messages', 'model' and 'stream') arguments to be given